In [2]:
import os
from resolve_api_samples.Examples.python_get_resolve import GetResolve
from pprint import pprint
import sys
from typing import List
from collections import Counter

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()

# media_path = '/Volumes/thom_7_for_Mac/tmp/素材'
media_path = '/Users/thom/Movies/Videos/Footages/悠长假期/素材'
sub_folders_full_path = media_storage.GetSubFolderList(media_path)
INVALID_EXTENSION = ["DS_Store", "JPG", "JPEG", "SRT"]  # TODO, 小写的情况还待考虑进去


def get_sub_folder_by_name(sub_folder_name: str):
    """Get folder object by name."""
    all_sub_folder = root_folder.GetSubFolderList()
    sub_folder_dict = {sub_folder.GetName(): sub_folder for sub_folder in all_sub_folder}
    return sub_folder_dict.get(sub_folder_name, "")


def absolute_file_paths(directory) -> list:
    absolute_file_path_list = []
    for directory_path, _, filenames in os.walk(directory):
        for f in filenames:
            absolute_file_path_list.append(os.path.abspath(os.path.join(directory_path, f)))
    return absolute_file_path_list


def import_clip() -> None:
    """Import footage from media storage into the corresponding sub-folder of the media pool root folder."""
    for cam_path in sub_folders_full_path:
        filename_and_fullpath_dict = {os.path.splitext(path)[0].replace(".", "").split('/')[-1]: path for path in
                                      absolute_file_paths(cam_path) if path.split('.')[-1] not in INVALID_EXTENSION}
        filename_and_fullpath_keys = list(filename_and_fullpath_dict.keys())
        filename_and_fullpath_keys.sort()
        filename_and_fullpath_value = [filename_and_fullpath_dict.get(i) for i in filename_and_fullpath_keys]
        current_folder = get_sub_folder_by_name(f"{cam_path.split('/')[cam_path.split('/').index('素材') + 1]}")
        media_pool.SetCurrentFolder(current_folder)
        media_storage.AddItemListToMediaPool(filename_and_fullpath_value)


import_clip()

In [25]:
footage_parent_path = '/Users/thom/Movies/Videos/Footages/tmp/素材'


def get_subfolder_name(source_media_fullpath: List[str]):
    subfolders_name = []
    for i in source_media_fullpath:
        split_fullpath = i.split('/')
        subfolders_name.append(split_fullpath[-1])
    return subfolders_name


class Project:
    def __init__(self) -> None:
        self.resolve = GetResolve()
        self.projectmanager = self.resolve.GetProjectManager()
        self.project = self.projectmanager.GetCurrentProject()
        self.media_storage = self.resolve.GetMediaStorage()
        self.media_pool = self.project.GetMediaPool()
        self.root_folder = self.media_pool.GetRootFolder()
        # self.timeline = self.project.GetCurrentTimeline()
        self.media_fullpath_list = media_storage.GetSubFolderList(footage_parent_path)

    def get_all_timeline(self) -> list:
        """
        Get all existing timelines.
        Return a list containing timeline object.
        """
        all_timeline = []
        for timeline_index in range(1, self.project.GetTimelineCount() + 1, 1):
            all_timeline.append(self.project.GetTimelineByIndex(timeline_index))
        return all_timeline

    def get_timeline_by_name(self, timeline_name: str):
        """Get timeline object by name."""
        all_timeline = self.get_all_timeline()
        timeline_dict = {timeline.GetName(): timeline for timeline in all_timeline}
        return timeline_dict.get(timeline_name, "")

    def get_subfolder_by_name(self, subfolder_name: str):
        """Get subfolders (folder object) under the root folder in the media pool."""
        all_subfolder = self.root_folder.GetSubFolderList()
        subfolder_dict = {subfolder.GetName(): subfolder for subfolder in all_subfolder}
        return subfolder_dict.get(subfolder_name, "")


project = Project()

'_Timeline'